<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'> Code Modules, Styling, and Helpful Functions</h1>

In [ ]:
%%html
<style> 
@import url('https://fonts.googleapis.com/css?family=Ewert|Roboto&effect=3d'); 
h1,h3 {text-shadow:2px 2px 2px #ff355e;} 
span {font-family:Roboto; color:midnightblue; text-shadow:4px 4px 4px #aaa;}
div.output_prompt {color:#ff00cc;}
div.input_prompt {color:#ff355e;} 
div.output_area pre,div.output_subarea {font-family:Roboto; font-size:15px; color:#ff355e;}
div.output_stderr pre {background-color:ghostwhite;}
</style>

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import os,h5py
import pandas as pd,numpy as np,xgboost as xgb
import pylab as pl,seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from keras.datasets import mnist
from keras.utils import to_categorical
path1='../input/digit-recognizer/'
path2='../input/classification-of-handwritten-letters/'

In [ ]:
def ohe(x): 
    return OneHotEncoder(categories='auto').fit(x.reshape(-1,1))\
           .transform(x.reshape(-1,1)).toarray().astype('int64')
def tts(X,y): 
    x_train,x_test,y_train,y_test=\
    train_test_split(X,y,test_size=.2,random_state=1)
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    return x_train,x_valid,x_test,y_train,y_valid,y_test

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'> Data Sets</h1>

In [ ]:
digits=pd.read_csv(path1+'train.csv')
digit_labels=digits['label'].values.astype('int16')
#cdigit_labels=ohe(digit_labels)
digit_images=digits.drop(labels=["label"],axis=1) 
digit_images=digit_images.values.astype('float32')/255
digit_images=(digit_images-.5)*2
[digit_labels.shape,digit_images.shape]

In [ ]:
x_train1,x_valid1,x_test1,\
y_train1,y_valid1,y_test1=tts(digit_images,digit_labels)
#y_train1=np.array([np.argmax(y) for y in y_train1])
#y_valid1=np.array([np.argmax(y) for y in y_valid1])
#y_test1=np.array([np.argmax(y) for y in y_test1])
[el.shape for el in [x_train1,x_valid1,x_test1,
                     y_train1,y_valid1,y_test1]]


In [ ]:
(x_train4,y_train4),(x_test4,y_test4)=mnist.load_data()
n=int(len(x_test4)/2)
x_valid4,y_valid4=x_test4[:n],y_test4[:n]
x_test4,y_test4=x_test4[n:],y_test4[n:]
#cy_train4=to_categorical(y_train4,10)
#cy_valid4=to_categorical(y_valid4,10)
#cy_test4=to_categorical(y_test4,10)
x_train4=x_train4.reshape(-1,784)
x_test4=x_test4.reshape(-1,784)
x_valid4=x_valid4.reshape(-1,784)
[el.shape for el in [x_train4,x_valid4,x_test4,
                     y_train4,y_valid4,y_test4]]

In [ ]:
f=h5py.File(path2+'LetterColorImages2.h5','r')
keys=list(f.keys())
letters=u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
letter_labels=np.array(f[keys[2]])
letter_images=np.array(f[keys[1]])/255
#cletter_labels=ohe(letter_labels)
gray_letter_images=np.dot(letter_images[...,:3],
                          [.299,.587,.114])
gray_letter_images=gray_letter_images.reshape(-1,32,32,1)
[letter_labels.shape,
 letter_images.shape,gray_letter_images.shape]

In [ ]:
x_train2,x_valid2,x_test2,\
y_train2,y_valid2,y_test2=tts(letter_images,letter_labels)
x_train3,x_valid3,x_test3,\
y_train3,y_valid3,y_test3=tts(gray_letter_images,letter_labels)
#ny_train2=np.array([np.argmax(y) for y in y_train2])
#ny_valid2=np.array([np.argmax(y) for y in y_valid2])
#ny_test2=np.array([np.argmax(y) for y in y_test2])
#ny_train3=np.array([np.argmax(y) for y in y_train3])
#ny_valid3=np.array([np.argmax(y) for y in y_valid3])
#ny_test3=np.array([np.argmax(y) for y in y_test3])
[el.shape for el in [x_train2,x_valid2,x_test2,
                     y_train2,y_valid2,y_test2,
                     x_train3,x_valid3,x_test3,
                     y_train3,y_valid3,y_test3]]

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>XGB Classifiers</h1>
### Digits

In [ ]:
params={'objective':'multi:softprob','verbosity':2,
        'random_state':42,'num_class':10,
        'n_estimators':784,'learning_rate':.3,'max_depth':11,
        'tree_method':'gpu_hist','predictor':'gpu_predictor'}
clf=xgb.XGBClassifier(**params)
clf.fit(x_train1,y_train1,eval_metric='mlogloss',
        eval_set=[(x_train1,y_train1),(x_valid1,y_valid1)])

In [ ]:
print(clf.evals_result())
y_xgb_train1=clf.predict(x_train1)
y_xgb_valid1=clf.predict(x_valid1)
y_xgb_test1=clf.predict(x_test1)
for [y,py] in [[y_train1,y_xgb_train1],
               [y_valid1,y_xgb_valid1],
               [y_test1,y_xgb_test1]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))

In [ ]:
params={'objective':'multi:softprob','verbosity':2,
        'random_state':42,'num_class':10,
        'n_estimators':784,'learning_rate':.3,'max_depth':11,
        'tree_method':'gpu_hist','predictor':'gpu_predictor'}
clf4=xgb.XGBClassifier(**params)
clf4.fit(x_train4,y_train4,eval_metric='mlogloss',
         eval_set=[(x_train4,y_train4),(x_valid4,y_valid4)])

In [ ]:
print(clf4.evals_result())
y_xgb_train4=clf4.predict(x_train4)
y_xgb_valid4=clf4.predict(x_valid4)
y_xgb_test4=clf4.predict(x_test4)
for [y,py] in [[y_train4,y_xgb_train4],
               [y_valid4,y_xgb_valid4],
               [y_test4,y_xgb_test4]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))

### Letters
Color Images

In [ ]:
clf2=xgb.XGBClassifier(objective="multi:softprob",verbosity=2,
                       random_state=42,learning_rate=.05,
                       max_depth=16,n_estimators=256,num_class=33,
                       tree_method='gpu_hist',predictor='gpu_predictor')
clf2.fit(x_train2.reshape(-1,32*32*3),y_train2,eval_metric='mlogloss',
         eval_set=[(x_train2.reshape(-1,32*32*3),y_train2),
                   (x_valid2.reshape(-1,32*32*3),y_valid2)])

In [ ]:
print(clf2.evals_result())
y_xgb_train2=clf2.predict(x_train2.reshape(-1,32*32*3))
y_xgb_valid2=clf2.predict(x_valid2.reshape(-1,32*32*3))
y_xgb_test2=clf2.predict(x_test2.reshape(-1,32*32*3))
for [y,py] in [[y_train2,y_xgb_train2],
               [y_valid2,y_xgb_valid2],
               [y_test2,y_xgb_test2]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))

Grayscaled Images

In [ ]:
clf3=xgb.XGBClassifier(objective="multi:softprob",verbosity=2,
                       random_state=42,max_depth=16,num_class=33,
                       tree_method='gpu_hist',predictor='gpu_predictor')
clf3.fit(x_train3.reshape(-1,32*32),y_train3,eval_metric='mlogloss',
         eval_set=[(x_train3.reshape(-1,32*32),y_train3),
                   (x_valid3.reshape(-1,32*32),y_valid3)])

In [ ]:
print(clf3.evals_result())
y_xgb_train3=clf3.predict(x_train3.reshape(-1,32*32))
y_xgb_valid3=clf3.predict(x_valid3.reshape(-1,32*32))
y_xgb_test3=clf3.predict(x_test3.reshape(-1,32*32))
for [y,py] in [[y_train3,y_xgb_train3],
               [y_valid3,y_xgb_valid3],
               [y_test3,y_xgb_test3]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))